### Building the model

For this project I'll be comparing the performace of two models, a Transformer Encoder from scratch  
and a pretrained BERT model that will be finetuned using the data

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
from transformers import TFAutoModel

We'll start by loading in our data and preprocessing it for our model

In [9]:
train_data = pd.read_csv('data/train_gr/train_clean.csv')
train_data.head()

,user_review,user_suggestion
0,im scared and hearing creepy voices so ill pa...,1
1,best game more better than sam peppers youtube...,1
2,a littly iffy on the controls but once you kno...,1
3,great game fun and colorful and all thata side...,1
4,not many games have the cute tag right next to...,1


In [11]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
train_data.user_review.dtype

dtype('O')